# 🐅 Парсер

Парсинг начинается со страниц, перечисленных в `pages.json`. Файл содержит необходимые сведения о структуре страниц сайта: стартовая страница, селекторы страницы, отвечающие за их описания (могут быть на стартовой `start_url` или внутренних страницах `event_urk`), количество соответствующих элементов. Если для ссылки события пока нет описания в файле `events.json`, в результате парсинга создаётся соответствующий объект. Этот файл используется для генерации карточек мероприятий в HTML, публикуемый на [matyushkin.github.io/events](https://matyushkin.github.io/events/).

In [ ]:
# soups.force_update_soups()

Специальный инструмент перезагрузки модулей:

In [47]:
import sys, importlib
modulenames = set(sys.modules) & set(globals())
allmodules = [sys.modules[name] for name in modulenames]
for module in allmodules:
    importlib.reload(module)

In [48]:
import math
import sys
import os
import importlib
import itertools
import json

import pandas as pd

# custom packages
import files
import soups
import urls
import langs
import handlers


def reload_modules():
    '''Rough technique to reload all imported modules'''
    modulenames = set(sys.modules) & set(globals())
    allmodules = [sys.modules[name] for name in modulenames]
    for module in allmodules:
        importlib.reload(module)

# Узнаем, какие из стартовых страниц, перечисленных в files.pages, размечены.
# Такие страницы содержат все поля, перечисленные в files.fields.
# Выводим список с помощью специальной функции pages_checked()

pages_checked = files.pages_checked()
print('Все необходимые поля имеются у следующих стартовых страниц: ')
for page in pages_checked:
    print(page)

# Для обработки полей, описывающих события, логична следующая последовательность:
# - cчитываемые поля со стартовой страницы (обычно это заголовк, дата, статус)
# - поля, расположенные на странице мероприятия (темы, докладчики, время начала)
# - получаемые из анализа предобработанных полей (теги, призы)


class StartPage:
    """Парсинг начальных страниц, описанных в pages.json"""
    def __init__(self, start_url):
        self.data, self.event_lists, self.events = {}, {}, {}
        self.data['start_url'] = start_url
        self.fields = files.fields_order(start_url)
        for field in self.fields['start']:
            self.event_lists[field] = files.get_content(self.data, field)
        event_urls = self.event_lists.pop('event_url')
        self.fields['start'].remove('event_url')
        for i, event_url in enumerate(event_urls):
            self.events[event_url] = {}
            for field in self.fields['start']:
                self.events[event_url][field] = self.event_lists[field][i]
    

    def __repr__(self):
        return self.actual_events
            
                
    @property
    def actual_events(self):
        """События, актуальные на текущий день"""
        events = self.events.copy()
        for event_url in self.events:
            if events[event_url]['date'] < handlers.current_date.isoformat():
                events.pop(event_url)
        return events


class EventPage:
    """Парсинг страниц событий"""
    def __init__(self, event_url, start_page):
        self.data = start_page.events[event_url]
        self.data['start_url'] = start_page.data['start_url']
        self.data['event_url'] = event_url
        
        for field in start_page.fields['event']:
            self.data[field] = files.get_content(self.data, field)
        for field in self.data:
            # удаление дупликатов в списках с сохранением порядка
            if type(self.data[field]) == list:
                self.data[field] = list(dict.fromkeys(self.data[field]))


for start_url in pages_checked:
    start_page = StartPage(start_url)
    print(f"Собираем данные о стартовой странице {self.data['start_url']}")
    actual = start_page.actual_events
    for event_url in actual:
        print(f'Получаем информацию о странице {event_url}')
        event = EventPage(event_url, start_page)
        files.events[event_url] = event.data

Все необходимые поля имеются у следующих стартовых страниц: 
https://events.yandex.ru/
https://it-events.com/


NameError: name 'self' is not defined

In [ ]:
# def get_content_test(start_url, event_url):
#     start_page = StartPage(start_url)
#     print(start_page.events)
#     actual = start_page.actual_events
#     event = EventPage(event_url, start_page)
#     print(event.data)

# get_content_test("https://events.yandex.ru/", "https://cloud.yandex.ru/events/143")

Удаляем устаревшие soup-объекты:

In [31]:
for key in soups.soups.keys()-actual:
    del soups.soups[key]

soups.write_soups(soups.soups)

In [ ]:
with open('files/events.json', 'w', encoding='utf-8') as events_file:
    json.dump(files.events, events_file, ensure_ascii=False)

# Анализируем и дополняем полученную информацию

Предварительно, если среди мероприятий имеются мероприятия, не обрабатываемые парсером (`events_special`) или рекламируемые (`events_promo`), обрабатываем их отдельно. Для этого помечаем их в датафрейме особым образом.

In [34]:
def promo_and_special():
    event_types = 'promo', 'special'
    d = {}
    for event_type in event_types:
        event_urls = list(eval(f'files.events_{event_type}.keys()'))
        for event_url in event_urls:
            d[event_url] = event_type
    return d


df = pd.DataFrame.from_dict(files.events, orient='index')

# Сразу выкидываем мероприятия, которые не хотим видеть
df = df.drop(list(files.events_bad.keys()), errors='ignore')

df = df.sort_values(by=['date'])
df = df[df['date'] >= handlers.current_date.isoformat()]   #! добавить проверку на время
df['type'] = df.index.map(promo_and_special())
df['tags'] = df.apply(lambda x: handlers.find_spec(x, 'tags'), axis=1)
df['types'] = df.apply(lambda x: handlers.find_spec(x, 'event_types'), axis=1)

# Для фильтрации мероприятий по месяцам, также метки "Позже" и "∞"
df['month'] = df.date.apply(langs.date_to_month)

Заполним NA-ячейки пустыми строками или пустыми списками:

In [35]:
def emptyfill(x, t):
    try:
        if math.isnan(x):
            if t == str:
                return ''
            elif t == list:
                return ['']
            else:
                return set()
        else:
            return x
    except:
        return x


for col in df:
    try:
        t = type(df[col][df[col].notnull()][0])
        df[col] = df[col].apply(lambda x: emptyfill(x, t))
    except IndexError:
        pass

In [37]:
df

,title,date,start_url,event_url,themes,time,description,speakers_companies,speakers,reg_url,online_status,organizers,location,price,start,event_type,type,tags,types,month
https://it-events.com/events/18717,Vertica 10. Новая версия - новые функции,2020-07-07,https://it-events.com/,https://it-events.com/events/18717,[],13:00,[Какие новые функции появились в Vertica 10? Р...,[],[],https://p.brighttalk.com/microfocus/ru/?commid...,Offline,[],,0,NaN,Вебинар,NaN,{Cloud},{},Июль
https://it-events.com/events/18768,Java Webinar,2020-07-07,https://it-events.com/,https://it-events.com/events/18768,[],11:00,[07 июля приглашаем на наш Java Webinar!\n \nВ...,[],[],https://events.epam.com/events/java-webinar,Offline,[],,0,NaN,Вебинар,NaN,{},{Вебинар},Июль
https://it-events.com/events/18792,Важность SEO на этапе разработки сайта,2020-07-07,https://it-events.com/,https://it-events.com/events/18792,[],15:00,"[Николай Новиков, SEO Specialist Netpeak Agenc...",[],[],https://bit.ly/2CUkpDJ,Offline,[],,0,NaN,Вебинар,NaN,{Business},{Вебинар},Июль
https://it-events.com/events/18798,Творческая онлайн-встреча «Пайплайн успешного ...,2020-07-07,https://it-events.com/,https://it-events.com/events/18798,[],19:00,[Приглашаем 7 июля на творческую онлайн-встреч...,[],[],https://bit.ly/3dSR0Xv,Online,[],,0,NaN,Практикум,NaN,{},{},Июль
https://it-events.com/events/18748,UX дизайн и архитектура,2020-07-07,https://it-events.com/,https://it-events.com/events/18748,[],19:00,"[На вебинаре мы поговорим о том, что такое UX ...",[],[],http:///itvdn.com/ru/webinars/description/ux-d...,Online,[],,0,NaN,Вебинар,NaN,{Business},{Вебинар},Июль
https://it-events.com/events/18778,Вернуть нельзя отказать. Специфика и тренды об...,2020-07-07,https://it-events.com/,https://it-events.com/events/18778,[],14:00,[Как учесть всё самое важное?\r\nКак автоматиз...,[],[],https://www.fnc-group.ru/news/kruglyy-stol-ver...,Offline,[],,0,NaN,Конференция,NaN,{},{},Июль
https://it-events.com/events/18661,Безопасность WEB: Уязвимости авторизации,2020-07-07,https://it-events.com/,https://it-events.com/events/18661,[],16:00,[Во вторник 7 июля 2020 пройдет бесплатный веб...,[],[],https://hacktory.clickmeeting.com/bezopasnost-...,Offline,[],,0,NaN,Вебинар,NaN,{},{Вебинар},Июль
https://it-events.com/events/18706,Предпринимательский форум «Pro_Таргетинг»,2020-07-07,https://it-events.com/,https://it-events.com/events/18706,[],12:00,[7 - 8 июля пройдет ежегодный предпринимательс...,[],[],https://mbm.mos.ru/education/obuchayushchiye-m...,Offline,[],,0,NaN,Конференция,NaN,{},{Конференция},Июль
https://cloud.yandex.ru/events/144,Вебинар. Как использовать управляемую базу дан...,2020-07-07,https://events.yandex.ru/,https://cloud.yandex.ru/events/144,[],15:00,"[Из вебинара вы узнаете, как устроен сервис Ya...",[Яндекс.Облако],[Всеволод Грабельников],https://cloud.yandex.ru/events/144#registration,Online,[Яндекс],,0,NaN,,NaN,{Cloud},{Вебинар},Июль
https://it-events.com/events/18723,Как банку перейти на безбумажный офис и сэконо...,2020-07-08,https://it-events.com/,https://it-events.com/events/18723,[],12:00,[На чем банку снижать издержки? За счет чего с...,[],[],https://clck.ru/PBruz,Online,[],,0,NaN,Конференция,NaN,{},{},Июль


# Собираем HTML-страницу и деплоим проект 💃

Для сборки страницы используем BeautifulSoup. Страницу собираем, объединяя soup-объекты в один.

In [ ]:
from bs4 import BeautifulSoup, Comment
from itertools import zip_longest
import copy

with open("files/event_card_template.html") as template:
    template = BeautifulSoup(template.read(), 'html.parser')


def add_info(template, data):
    s = copy.copy(template)
    
    # Информационный блок
    data_block = s.find("div", {"class":"event"})
    companies = [*data.organizers, *data.speakers_companies]
    companies = " ".join(list(set(companies))).strip()
    data_block['data-companies'] = companies
    
    data_block['data-month'] = data.month
    data_block['data-price'] = data.price
    data_block['data-online'] = data.online_status
    if data.tags:
        data_block['data-tags'] = ", ".join(data.tags)
    if data.types:
        data_block['data-types'] = ", ".join(data.types)
    if data.type == 'promo':
        data_block['class'].append('promo')
    
    # Обработка заголовка
    header = s.find("h2")
    header.string = data.title
    
    # Блок с описанием места и времени
    time_and_space = s.find("p", {"class":"time_and_space"})
    s.time.string = langs.make_datetime_string(data.date, data.time)
    s.time.wrap(s.new_tag('a', attrs={'href':data.event_url}))
    if data.reg_url:
        time_and_space.append(', ')
        reg_url = s.new_tag('a', attrs={'href':data.reg_url})
        reg_url.string = 'регистрация'
        time_and_space.append(reg_url) 
    time_and_space.append('.')
    
    # Блок с темами и докладчиками
    themes_and_speakers = s.find("p", {"class":"themes_and_speakers"})
    
    if data.themes != ['']:
        for theme in data.themes:
            if theme:
                line = s.new_tag('li')
                line.string = theme
                s.ul.append(line)
    if data.speakers != ['']:
        if len(data.speakers) == 1:
            themes_and_speakers.append(f'👤 {data.speakers[0]}.')
        else:
            speakers_str = ', '.join(data.speakers)
            themes_and_speakers.append(f'👥 {speakers_str}.')
    if data.themes == [''] and data.speakers != ['']:
        s.ul.extract()
    if data.themes == [''] and data.speakers == ['']:
        s.find("p", {"class":"themes_and_speakers"}).extract()
    
    # Блок с описанием мероприятия
    description = s.find("p", {"class":"description"})
    description.string = data.description[0]
    
    # Удаляем комментарии
    for element in s(text = lambda text: isinstance(text, Comment)):
        element.extract()

    return s

df['soup'] = df.apply(lambda x: add_info(template, x), axis=1)
df['html'] = df['soup'].apply(lambda x: str(x))
total = ''.join(df['html'])

total += '''<div class="filter__msg" hidden>
<p>🕵 К сожалению, для выбранных фильтров пока
нет ни одного подходящего мероприятия.</p></div>'''

path_to_html_template = "../mgio/11ty/_includes/events.njk"
with open(path_to_html_template, 'w', encoding='utf-8') as html_template:
    html_template.write(f'{total}')

Используем тот же подход для сборки меню фильтров. 

In [ ]:
import langs

all_tags = set(itertools.chain.from_iterable(df.tags))
all_types = set(itertools.chain.from_iterable(df.types))
all_locs = set(itertools.chain.from_iterable(df.location))
all_months = langs.month_names_for_time_filters()[1:] + ['Позже', '∞']
#all_types = set(itertools.chain.from_iterable(df.type))

with open("files/event_filter_panel_template.html") as filter_template:
    filter_soup = BeautifulSoup(filter_template.read(), 'html.parser') 


def filter_item_append(name, value):
    s = filter_soup.find('div', {'class':f'filter__container--{name}'}).div
    t = filter_soup.new_tag('div')
    t['class'] = 'filter__item filter__button'
    t.string = value
    s.append(t)
    

for tag in all_tags:
    filter_item_append('tag', tag)

for type_ in all_types:
    filter_item_append('type', type_)

for month in all_months:
    filter_item_append('month', month)

for loc in all_locs:
    filter_item_append('location', loc)

    
with open("../mgio/11ty/_includes/events_filter.njk", 'w', encoding='utf-8') as html_template:
    html_template.write(str(filter_soup))
    
!cd ../mgio/11ty/; npx eleventy --passthroughall --output=../../matyushkin.github.io
!cd ../matyushkin.github.io/; rm -rf 404
!cd ../matyushkin.github.io/; rm -rf README


# посмотрим, что получилось в браузере
import webbrowser
url = "../matyushkin.github.io/events/index.html"
webbrowser.open_new_tab(url)

In [ ]:
!cd ../matyushkin.github.io/; git add . ; git commit -m "Events type filter is added"; git push origin master

In [ ]:
for line in total.split('.'):
    if sentence.lower() in line and <

In [ ]:
files.events_bad['sentences']